# Notebook to extract hidden-states and attention heads activations from LSTM model predictions

In [1]:
import os
import glob
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from model import LSTMExtractor
from tokenizer import tokenize
from utils import set_seed
from data import Dictionary
from utils import read_yaml, save_yaml

In [2]:
def check_folder(path):
    """Create adequate folders if necessary."""
    try:
        if not os.path.isdir(path):
            check_folder(os.path.dirname(path))
            os.mkdir(path)
    except:
        pass

Defining variables:

In [3]:
template = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/text_english_run*.txt' # path to text input
language = 'english'

In [4]:
name_template = 'weights_{}_embedding-size_{}_nhid_{}_nlayers_{}_dropout_{}_wiki_kristina_english.pt'

In [5]:
rnn_types = ['LSTM', 'LSTM']
ninps = ['650', '600']
nhids =  ['650', '300']
nlayers = ['2', '1']
dropouts =  ['02', '02']
vocab_path = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training'
config_path_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/models/LSTM/configs/'
trained_model_folder = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/'
path_to_data = '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations'

In [6]:
#template = '/Users/alexpsq/Code/Parietal/data/text_english_run*.txt' # path to text input
#config_path_folder = '/Users/alexpsq/Code/Parietal/data/configs/'
#trained_model_folder = '/Users/alexpsq/Code/Parietal'
#path_to_data = '/Users/alexpsq/Code/data/stimuli-representations'
#vocab_path = '/Users/alexpsq/Code/data/'

In [7]:
pretrained_lstm_models = [
    trained_model_folder + name_template.format(rnn_type, ninp, nhid, nlayer, dropout) for (rnn_type, ninp, nhid, nlayer, dropout) in zip(rnn_types, ninps, nhids, nlayers, dropouts)] # path to the model from which we want to retrieve the activations
infos = [os.path.basename(model).split('_') for model in pretrained_lstm_models]
names = ['_'.join(os.path.basename(model).split('.')[0].split('_')[1:]) for model in pretrained_lstm_models]
config_paths = [os.path.join(config_path_folder, 'config_' + name + '.yml') for name in names]
saving_path_folders = [
    os.path.join(path_to_data, '{}/{}'.format(language, name)) for name in names]
prediction_types = ['sentence' for i in pretrained_lstm_models]

In [8]:
def get_config(rnn_type='LSTM',
               language='english',
               ntoken=50001,
               ninp=650,
               nhid=650,
               nlayers=2,
               dropout='02',
               tie_weights=False,
               eos_separator='<eos>',
               cuda=True,
               weights_path=None,
               path_to_vocab=None,
               includ_surprisal=True,
               includ_entropy=True,
               parameters=['in', 'forget', 'out', 'c_tilde', 'hidden', 'cell']):
    config_template = {
        'rnn_type': rnn_type,
        'language': language,
        'ntoken': ntoken,
        'ninp': ninp,
        'nhid': nhid,
        'nlayers': nlayers,
        'dropout': int(dropout)/10,
        'tie_weights': tie_weights,
        'eos_separator': eos_separator,
        'cuda': cuda,
        'weights_path': os.path.join(weights_path, name_template.format(rnn_type, ninp, nhid, nlayers, dropout)),
        'path_to_vocab': path_to_vocab,
        'includ_surprisal': includ_surprisal,
        'includ_entropy': includ_entropy,
        'parameters': parameters}
    return config_template


In [9]:
for index, (rnn_type, ninp, nhid, nlayer, dropout) in enumerate(zip(rnn_types, ninps, nhids, nlayers, dropouts)):
    config_template =  get_config(rnn_type=rnn_type,
                                   language='english',
                                   ntoken=50001,
                                   ninp=int(ninp),
                                   nhid=int(nhid),
                                   nlayers=int(nlayer),
                                   dropout=dropout,
                                   weights_path=trained_model_folder,
                                   path_to_vocab=vocab_path)
    save_yaml(config_template, config_paths[index])

In [10]:
config_template


{'rnn_type': 'LSTM',
 'language': 'english',
 'ntoken': 50001,
 'ninp': 600,
 'nhid': 300,
 'nlayers': 1,
 'dropout': 0.2,
 'tie_weights': False,
 'eos_separator': '<eos>',
 'cuda': True,
 'weights_path': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/models/english/weights_LSTM_embedding-size_600_nhid_300_nlayers_1_dropout_02_wiki_kristina_english.pt',
 'path_to_vocab': '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/text/english/lstm_training',
 'includ_surprisal': True,
 'includ_entropy': True,
 'parameters': ['in', 'forget', 'out', 'c_tilde', 'hidden', 'cell']}

Creating iterator for each run:

In [11]:
paths = sorted(glob.glob(template))

In [12]:
vocab = Dictionary(vocab_path, language)

In [13]:
iterator_list = [tokenize(path, language, train=False, vocab=vocab) for path in paths]

  0%|          | 0/135 [00:00<?, ?it/s]

Tokenizing...
Preprocessing...
Preprocessed.
Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


 36%|███▌      | 63/176 [00:00<00:00, 624.09it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


 34%|███▍      | 59/173 [00:00<00:00, 583.47it/s]

...

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


 30%|██▉       | 62/207 [00:00<00:00, 612.80it/s]

Tokenized.
Tokenizing...
Preprocessing...
Preprocessed.


100%|██████████| 207/207 [00:00<00:00, 858.66it/s]

Tokenized.


## Activation extraction

In [24]:
for index, config in enumerate(config_paths):
    extractor = LSTMExtractor(config, language, names[index], prediction_types[index], output_hidden_states=True)
    print(extractor.name, ' - Extracting activations ...')
    for run_index, iterator in tqdm(enumerate(iterator_list)):
        print("############# Run {} #############".format(run_index))
        check_folder(saving_path_folders[index])
        activations  = extractor.extract_activations(iterator, language)
    
        activations.to_csv(os.path.join(saving_path_folders[index], 'activations_run{}.csv'.format(run_index + 1)), index=False)
        
        

0it [00:00, ?it/s]
  0%|          | 9/1894 [00:00<00:21, 87.50it/s]

LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_02  - Extracting activations ...
############# Run 0 #############



 67%|██████▋   | 1270/1894 [00:18<00:12, 48.29it/s]


100%|██████████| 1894/1894 [00:30<00:00, 62.26it/s]
1it [00:49, 49.08s/it]

############# Run 1 #############


...


100%|██████████| 2435/2435 [00:09<00:00, 263.63it/s]
7it [01:31, 13.45s/it]
  0%|          | 0/2038 [00:00<?, ?it/s]

############# Run 7 #############



100%|██████████| 2038/2038 [00:07<00:00, 286.42it/s]
8it [01:43, 12.96s/it]
  0%|          | 0/2530 [00:00<?, ?it/s]

############# Run 8 #############



100%|██████████| 2530/2530 [00:09<00:00, 273.02it/s]
9it [01:58, 13.16s/it]


In [25]:
names

['LSTM_embedding-size_650_nhid_650_nlayers_2_dropout_02_wiki_kristina_english',
 'LSTM_embedding-size_600_nhid_300_nlayers_1_dropout_02_wiki_kristina_english']